In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

In [2]:
url = 'https://www.quora.com/What-is-time-in-space'

In [3]:
def extractQuestionTitle(driver):
    try:
        question_title = driver.find_element_by_xpath('.//span[contains(@class, "ui_qtext_rendered_qtext")]').text
    except Exception as e:
        question_title = 'error'
    return question_title

In [4]:
def extractTopics(driver):
    topic_list = []
    
    try:
        topic_area = driver.find_element_by_xpath(".//div[contains(@class,'QuestionTopicHorizontalList')]")
        
    except:
        return topic_list
    
    try:
        topics = topic_area.find_elements_by_xpath(".//span[@class = 'TopicName TopicNameSpan']")
        for topic in topics:
            topic_list.append(topic.text)
    except:
        topic_list = ['error']
           
    return topic_list

In [5]:
def scrollDownPage(driver, speed):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [6]:
def extractRespondContent(driver):
    try:
        respond_content = driver.find_element_by_xpath(".//div[@class = 'q-text']").text
    except Exception as e:
        repond_content = 'error'
        
    return respond_content

In [7]:
def extractRespondViewAndUpvote(driver):
    try:
        respond_stat = driver.find_element_by_xpath(".//div[@class = 'q-text qu-mt--small qu-color--gray_light qu-fontSize--small qu-passColorToLinks']").text

        view_text = respond_stat.split(' view')[0]
        if 'k' in view_text:
            respond_view = int(float((view_text.split('k'))[0]) * 1000)
        elif 'm' in view_text:
            respond_view = int(float((view_text.split('m'))[0]) * 1000000)
        else:
            respond_view = int(view_text)

        if 'Upvoter' in respond_stat:
            upvote_text = (respond_stat.split('View ')[1]).split(' ')[0]
            if 'k' in upvote_text:
                respond_upvote = int(float((upvote_text.split('k'))[0]) * 1000)
            elif 'm' in upvote_text:
                respond_upvote = int(float((upvote_text.split('m'))[0]) * 1000000)
            else:
                respond_upvote = int(upvote_text)
        else:
            respond_upvote = 0

    except:
        respond_view = 0
        respond_upvote = 0
        
    return respond_view, respond_upvote

In [8]:
def extractDate(driver):
    try:
        log = driver.find_element_by_xpath(".//p[@class = 'log_action_bar']")
        date = log.text.split('· ')[1]
    except:
        date = 'error'
    
    return date

In [9]:
def extractAuthor(driver):
    block_header = driver.find_element_by_xpath (".//div[@class = 'feed_item_activity']")
    try:
        user_name = block_header.find_element_by_xpath(".//span[@class = 'anon_user']").text
    except Exception:
        user_name = block_header.find_element_by_xpath(".//a[@class = 'user']").text
    except Exception as e:
        user_name = 'error'
    
    return user_name

In [10]:
def log_stat_extraction(driver, driver2):
    # return element
    respond_author_list = []
    respond_content_list = []
    respond_upvote_list = []
    respond_view_list = []
    respond_date_list = []
    respond_count = 0
    question_auther = ''
    question_create_time = ''
    
    try:
        block_list = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")
    except:
        respond_author_list =  ['error']
        respond_content_list = ['error']
        respond_upvote_list = ['error']
        respond_view_list = ['error']
        respond_date_list = ['error']
        respond_count = 0
        question_auther = 'error'
        question_create_time = 'error'
        
        return respond_author_list, respond_content_list, respond_upvote_list, respond_view_list, respond_date_list, respond_count, question_auther, question_create_time
        
    for block in block_list:
        block_error = []
        
        block_header = block.find_element_by_xpath (".//div[@class = 'feed_item_activity']").text
        
        try:
            block_revision = block.find_element_by_xpath (".//div[@class = 'revision']").text
        except:
            continue
            
        if 'answer has since been deleted' in block_revision:
            continue
        elif 'deleted' in block_header:
            continue
        
        if 'Answer' in block_header:
            # click the answer link
            try:
                url2 = block.find_element_by_xpath (".//div[@class = 'feed_item_activity']/a").get_attribute('href')
                driver2.get(str(url2))
                respond_count += 1
            except Exception as e:
                continue
                
            # extract author name(log page and driver 1)
            respond_author = extractAuthor(block)
            respond_author_list.append(respond_author)
            
            # extract content
            respond_content = extractRespondContent(driver2)
            respond_content_list.append(respond_content)
                
            # extract views and upvote
            respond_view, respong_upvote = extractRespondViewAndUpvote(driver2)
            respond_view_list.append(respond_view)
            respond_upvote_list.append(respong_upvote)
                
            # extract date(log page and driver 1)
            respond_date = extractDate(block)
            respond_date_list.append(respond_date)
            
        elif 'Question added by' in block_header:
            # extract author name
            question_author = extractAuthor(block)
            
            # extract date
            question_create_time = extractDate(block)

    return respond_author_list, respond_content_list, respond_upvote_list, respond_view_list, respond_date_list, respond_count, question_author, question_create_time



In [11]:
def extractNoOfFollowerAndView(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_follower = int((stats[0].text.split(' '))[0])
        no_of_view = (stats[1].text.split(' '))[0]
    except:
        no_of_follower = 0
        no_of_view = 0
        
    return no_of_follower, no_of_view

In [12]:
def extractQuestion(driver, driver2, url):
    data = {}

    driver.get(url+'/log')
    data['question_title'] = extractQuestionTitle(driver)
    data['topic_list'] = extractTopics(driver)
    
    scrollDownPage(driver, speed = 5)
    data['name_list'], data['ans_list'], data['upvote_list'], data['view_list'], data['date_list'], data['respond_collected'], data['question_author'], data['create_time'] = log_stat_extraction(driver, driver2)
    
    data['no_of_follower'], data['no_of_view'] = extractNoOfFollowerAndView(driver)
    
    for key, value in data.items():
        print(value)

In [13]:
driver = webdriver.Chrome(executable_path='./chromedriver')
driver2 = webdriver.Chrome(executable_path='./chromedriver')

In [14]:
extractQuestion(driver, driver2, url)

What is time in space?
['Spacetime', 'Time (physics)', 'Physics']
['Jess H. Brewer', 'Alfred Dominic Vella']
["I like Alfred's first suggestion. You have asked a question that can be interpreted in many ways, but none with enough clarity to make a serious answer possible.", 'Perhaps in the future when someone is bad they get locked away on some penal colony far away?\nIf you mean how space and time fit together, then it appears that they are not so different in that by changing our speed relative to others we can tilt or rotate our space into their time and vice versa.\nOf course they are not exactly the same because the tilting can only go so far being limited to the speed of light.']
[0, 1]
[323, 141]
['Mar 11 2016', 'Mar 10 2016']
2
Yosef Alzubaidi
Mar 10 2016
11
2,686


In [15]:
driver.quit()
driver2.quit()